In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import json

In [2]:
def configure_ipv4_subnet(address_list):
    try:
        subnet = IPv4Network(address_list)
        available_ips = list(subnet)[1:]
    except Exception as e:
        print(f"Exception: {e}")
    return subnet, available_ips

In [3]:
def get_slice_by_name_or_id(slice_name=None, slice_id=None):
    if slice_name is None and slice_id is None:
        slice = None
    elif slice_name is not None:
        slice = fablib.get_slice(name=slice_name)
    else:
        slice = fablib.get_slice(slice_id=slice_id)
    return slice

In [4]:
def get_slice_ssh_commands(fablib, slice_name=None, slice_id=None):
    slice = get_slice_by_name_or_id(slice_name, slice_id)
    nodes = slice.get_nodes()
    node_hostnames, node_ssh_commands = [], []
    ssh_command_prefix = "ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config "
    for node in nodes:
        try:
            node_username, node_management_ip = str(node.get_username()), str(node.get_management_ip())
            node_hostnames.append(node_username + "@" + node_management_ip)
            node_ssh_commands.append(ssh_command_prefix + node_hostnames[-1])
            print("Node {}:".format(len(node_hostnames)), node.get_name())
            print("Node Hostname:", node_hostnames[-1])
            print("SSH Access Command:", node_ssh_commands[-1])
            print()
        except Exception as e:
            print("Exception in Parsing Node Details:", node, e)

    # print("Node Host Names:", node_hostnames)
    # print("Node SSH Commands:", node_ssh_commands)

In [5]:
def get_slice_components(fablib, slice_name=None, slice_id=None):
    slice = get_slice_by_name_or_id(slice_name, slice_id)
    nodes = slice.get_nodes()
    for node in nodes:
        components = node.get_components()
        print(vars(components[0]))
    
    
def get_slice_interfaces(fablib, slice_name=None, slice_id=None):
    slice = get_slice_by_name_or_id(slice_name, slice_id)
    nodes = slice.get_nodes()
    interfaces = slice.get_interfaces()
    for interface in interfaces:
        interface_vals = vars(interface)
        print(interface_vals["network"])

In [6]:
def configure_node_interface(node, network_name, subnet, available_ips):
    node_interface = node.get_interface(network_name=network_name)
    print("Node Interface:", node_interface)
    node_address = available_ips.pop(0)
    print("Node Address:", node_address, "for Node:", node.get_name())
    node_interface.ip_addr_add(addr=node_address, subnet=subnet)
    return node_address

def unconfigure_node_interface(node, network_name, subnet, ip_address):
    node_interface = node.get_interface(network_name=network_name)
    print("Node Interface:", node_interface)
    node_interface.ip_addr_del(addr=ip_address, subnet=subnet)
    print("Deleted IP Address:", ip_address, "for Node:", node.get_name())

In [8]:
fablib = fablib_manager()
slice_name = "Kiran_Integrated_Test_1"
get_slice_ssh_commands(fablib, slice_name=slice_name)
# get_slice_components(fablib, slice_name=slice_name) 
# get_slice_interfaces(fablib, slice_name=slice_name)

Node 1: h1
Node Hostname: ubuntu@129.114.110.87
SSH Access Command: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.87

Node 2: s1
Node Hostname: ubuntu@129.114.110.125
SSH Access Command: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.125

Node 3: h2
Node Hostname: ubuntu@129.114.110.103
SSH Access Command: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.103

Node 4: h3
Node Hostname: ubuntu@129.114.110.83
SSH Access Command: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.83

Node 5: _meas_node
Node Hostname: ubuntu@129.114.110.75
SSH Access Command: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.75



In [10]:
slice = get_slice_by_name_or_id(slice_name, slice_id=None)
nodes = slice.get_nodes()
h1, s1, h2, h3, _ = nodes

net1 = "C1"
address_list = "192.168.1.0/24"
subnet, available_ips = configure_ipv4_subnet(address_list=address_list)
s1_i1_address = configure_node_interface(s1, net1, subnet, available_ips)
h1_address = configure_node_interface(h1, net1, subnet, available_ips)

net2 = "C2"
address_list = "192.168.2.0/24"
subnet, available_ips = configure_ipv4_subnet(address_list=address_list)
s1_i2_address = configure_node_interface(s1, net2, subnet, available_ips)
h2_address = configure_node_interface(h2, net2, subnet, available_ips)

net3 = "C3"
address_list = "192.168.3.0/24"
subnet, available_ips = configure_ipv4_subnet(address_list=address_list)
s1_i3_address = configure_node_interface(s1, net3, subnet, available_ips)
h3_address = configure_node_interface(h3, net3, subnet, available_ips)

Node Interface: ---------------------  --------------------------
Name                   s1-switch_nic_1_conn_h1-p1
Network                C1
Bandwidth              0
VLAN
MAC                    3A:8A:C8:9F:8D:57
Physical OS Interface  ens10
OS Interface           ens10
---------------------  --------------------------
Node Address: 192.168.1.1 for Node: s1
Node Interface: ---------------------  -----------------
Name                   h1-host_nic_1-p1
Network                C1
Bandwidth              0
VLAN
MAC                    32:A2:E4:8E:94:EA
Physical OS Interface  ens8
OS Interface           ens8
---------------------  -----------------
Node Address: 192.168.1.2 for Node: h1
Node Interface: ---------------------  --------------------------
Name                   s1-switch_nic_1_conn_h2-p1
Network                C2
Bandwidth              0
VLAN
MAC                    32:D1:AA:31:AD:08
Physical OS Interface  ens7
OS Interface           ens7
---------------------  ------------------

In [ ]:
unconfigure_node_interface(h1, net1, subnet, h1_address)
unconfigure_node_interface(h2, net2, subnet, h2_address)
unconfigure_node_interface(h3, net3, subnet, h3_address)
unconfigure_node_interface(s1, net1, subnet, s1_i1_address)
unconfigure_node_interface(s1, net2, subnet, s1_i2_address)
unconfigure_node_interface(s1, net3, subnet, s1_i3_address)